In [ ]:
import glob
from CustomResnet50 import CustomResNet50

figuratif_data = glob.glob('./Dataset/figuratif/**/*.jpg', recursive=True)
abstrait_data = glob.glob('./Dataset/abstrait-v2/**/*.jpg', recursive=True)

print(f'{len(figuratif_data)} imágenes en figuratif')
print(f'{len(abstrait_data)} imágenes en abstrait')


6399 imágenes en figuratif
1315 imágenes en abstrait


In [ ]:
import glob
import torch
from CustomResnet50 import CustomResNet50
from dataloaders import make_dataloaders

figuratif_data = glob.glob('./Dataset/figuratif/**/*.jpg', recursive=True)
abstrait_data  = glob.glob('./Dataset/abstrait-v2/**/*.jpg', recursive=True)

print(f'{len(figuratif_data)} imágenes en figuratif')
print(f'{len(abstrait_data)} imágenes en abstrait')

train_loader, val_loader, test_loader = make_dataloaders(
    figuratif_data, abstrait_data,
    batch_size=32,
    val_split=0.2,
    test_split=0.1,
    random_seed=123
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CustomResNet50(device, save_path="./checkpoints/mejor.pth", n_classes=2)

counts = torch.tensor([1315,6399],dtype=torch.float)
class_weights = 1.0 / counts
class_weights= class_weights.float()
class_weights = class_weights.to(device)



criterion = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = torch.optim.SGD(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3, momentum=0.9
)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

history = model.fit(
    train_loader, val_loader,
    criterion, optimizer,
    n_epochs=20,
    scheduler=scheduler
)

test_loss, test_acc = model.test_model(test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")


6399 imágenes en figuratif
1315 imágenes en abstrait


KeyboardInterrupt: 